In [19]:
print("=== PROCEDURE, REFERENCE, & LIMITATIONS ===")
print("Procedure: Direct Design Method")
print("Reference: ACI 318M-14 | RCD by Darwin & Nelson")
print("Limitations: \n minimum 3x3 panel. If fewer, interior negative moments tend to be too small. \n Ratio not greater than 2. \n Span lengths not differ by more than 1/3 of the longer span. \n applicable only to braced frames \n Service live load shall not exceed two times the service dead load. \n relative stiffness of beams in two perpedicular directions shall not be lt 0.2 or gt 5.0.")

=== PROCEDURE, REFERENCE, & LIMITATIONS ===
Procedure: Direct Design Method
Reference: ACI 318M-14 | RCD by Darwin & Nelson
Limitations: 
 minimum 3x3 panel. If fewer, interior negative moments tend to be too small. 
 Ratio not greater than 2. 
 Span lengths not differ by more than 1/3 of the longer span. 
 applicable only to braced frames 
 Service live load shall not exceed two times the service dead load. 
 relative stiffness of beams in two perpedicular directions shall not be lt 0.2 or gt 5.0.


In [20]:
import math
pi = math.pi
def sqrt(x):
    return math.sqrt(x)
def rd(x):
    return round(x,2)
def roundup(x):
    return math.ceil(x/10.0) * 10

In [21]:
#conversions
def kPa_to_psf(x):
    return round(x*144/6.89655172413793,2)
def psf_to_kPa(x):
    return round(x*6.89655172413793/144,2)
def kNm_to_kipft(x):
    """
    Output:
    Converts kN-m to kip-ft
    """
    return rd(x/1.35582)
def kipft_to_kNm(x):
    """
    Output:
    Converts kip-ft to kN-m
    """
    return rd(x*1.35582)

In [22]:
#Class and methods
class Beam:
    def __init__ (self, name, dimy, dimz, length):
        self.name = name
        self.dimy = dimy
        self.dimz = dimz
        self.length  = length
    def Icb(self):
        return self.dimy*self.dimz**3/12
class Slab:
    def __init__ (self, name, L, W, thk):
        self.name = name
        self.L = L #The larger dimension
        self.W = W 
        self.thk = thk
    def Ics(self):
        return self.W*self.thk**3/12

In [23]:
#Slab and Beam Material Properties
fc = 27 #MPa
Ec = rd(4700*sqrt(fc)/1000) #GPa, for normal weight concrete
fy = 230 #MPa
print("=== MATERIAL PROPERTIES ===")
print(f"fc = {fc}MPa | fy = {fy}MPa | Ec = {Ec}GPa")

#Beam Material and Geometric Properties
B_north = Beam("B_north",300,600,7700)
B_south = Beam("B_south",300,600,7700)
B_east = Beam("B_east",300,600,7700)
B_west = Beam("B_west",300,600,7700)

#Slab Material and Geometric Properties
print("=== GEOMETRIC PROPERTIES ===")
S1 = Slab("S1",7260,5730,150)
lb = S1.L #larger dimension
la = S1.W #shorter dimension
print(f"Initial Slab Geometric Properties\n L = {S1.L}mm, W = {S1.W}mm, thk = {S1.thk}mm.")
beta = S1.L / S1.W #Ratio of longer to shorter slab dim
print(f"Ratio of longer to shorter slab clear span: {rd(beta)}")

#Beam-to-slab Stiffness Ratio, alpha_f
alpha_f = B_north.Icb() / S1.Ics()
print(f"Average Relative Stiffness beam to slab, alpha_f = {rd(alpha_f)}.")


=== MATERIAL PROPERTIES ===
fc = 27MPa | fy = 230MPa | Ec = 24.42GPa
=== GEOMETRIC PROPERTIES ===
Initial Slab Geometric Properties
 L = 7260mm, W = 5730mm, thk = 150mm.
Ratio of longer to shorter slab clear span: 1.27
Average Relative Stiffness beam to slab, alpha_f = 3.35.


In [24]:
#Slab Minimum Thickness per 8.3.1.2 ACI318M-14
print("=== MIN. THICKNESS (8.3.1.2) ===")
if alpha_f > 0.20 and alpha_f < 2:
    print(f"1. case 1 governs: 0.20 < alpha_f < 2.0")
    h_min = rd(max( S1.L * (0.8 + fy/1400) / (36+5*(S1.L/S1.W)*(alpha_f - 0.2)) , 125)) #mm
else: 
    print(f"1. case 2 governs: alpha_f > 2.0")
    h_min = rd(max( S1.L * (0.8 + fy/1400) / (36+9*(S1.L/S1.W)) , 90)) #mm
    
print(f"2. Minimum Slab Thickness (ACI318M-14, 8.3.1.2) = {h_min}mm.")
S1.thk = roundup(max(S1.thk,h_min))
print(f"3. Adopt S1.thk = {S1.thk}mm")
#Factored Uniform Load
print("=== APPLIED LOADS ===")
DL = rd(24*max(h_min,S1.thk)*.001 + 0.2) #selfweight of slab, kPa
LL = 7.0 #kPa
qa = DL + LL
qu = 1.2*DL + 1.6*LL
print(f"1. DL = {DL}kPa ({kPa_to_psf(DL)}psf) | LL = {LL}kPa ({kPa_to_psf(LL)}psf)")
print(f"2. Service Uniform Pressure, qa = {rd(qa)}kPa ({kPa_to_psf(qa)}psf)")
print(f"3. Factored Uniform Pressure, qu = {rd(qu)}kPa ({kPa_to_psf(qu)}psf)") #kPa

=== MIN. THICKNESS (8.3.1.2) ===
1. case 2 governs: alpha_f > 2.0
2. Minimum Slab Thickness (ACI318M-14, 8.3.1.2) = 147.68mm.
3. Adopt S1.thk = 150mm
=== APPLIED LOADS ===
1. DL = 3.8kPa (79.34psf) | LL = 7.0kPa (146.16psf)
2. Service Uniform Pressure, qa = 10.8kPa (225.5psf)
3. Factored Uniform Pressure, qu = 15.76kPa (329.07psf)


In [25]:
#Minimum reinforcement near tension face for each dir
print("=== MIN. REINF. NEAR TENSION FACE, EA.DIR. (8.6.1) ===")
if fy < 420:
    Asmin = rd(0.0020 * 1000 * S1.thk)
    print(f"1. Asmin = 0.0020Ag = {Asmin}mm2 per mtr width of slab")
else:
    Asmin = rd( max( 0.0018*420*1000*S1.thk/fy , 0.0014*1000*S1.thk ) )
    print(f"1. Asmin = {Asmin}mm2 per mtr width of slab")

#Minimum spacing of flexural reinforcement at crit sections
print("1. Max. spacing of flex. reinf. at crit. sec. must not exceed 2*slab_thk per 8.7.3")
s_max = int(2*S1.thk) #mm
print(f"2. s_max = {s_max}mm")

#Corner restraint in slab
print("=== CORNER RESTRAINT IN SLAB (8.7.3) ===")
lcor = int(rd(lb / 5)) #mm
print(f"1. Length corner bar, lcor = {lcor}mm")
Mu_cor = rd((1/2)*qu*(lcor*0.001)**2) #kN-m
print(f"2. Mu_cor = {Mu_cor}kN-m")


=== MIN. REINF. NEAR TENSION FACE, EA.DIR. (8.6.1) ===
1. Asmin = 0.0020Ag = 300.0mm2 per mtr width of slab
1. Max. spacing of flex. reinf. at crit. sec. must not exceed 2*slab_thk per 8.7.3
2. s_max = 300mm
=== CORNER RESTRAINT IN SLAB (8.7.3) ===
1. Length corner bar, lcor = 1452mm
2. Mu_cor = 16.61kN-m


In [26]:
#Case: Simply Supported Square Slab, if only bending moments were present
print("=== CASE: SIMPLY SUPPORTED SQUARE SLAB ===")
print("1. Twisting moment relieves 25 percent of bending moment.")
print("2. Bending moment formula: (qu/2)*l**2 / 8 less 25%")
Mu_square = ( ((qu*0.5)*(lb*0.001)**2) / 8 ) * 0.75 #kN-m
print(f"3. Mu_squareslab = {rd(Mu_square)}kN-m per mtr width")

=== CASE: SIMPLY SUPPORTED SQUARE SLAB ===
1. Twisting moment relieves 25 percent of bending moment.
2. Bending moment formula: (qu/2)*l**2 / 8 less 25%
3. Mu_squareslab = 38.94kN-m per mtr width


In [27]:
#Total Statical Moment, Mo (CONSIDER BENDING OF THE LONGER SPAN)
print("=== TOTAL STATICAL MOMENT, Mo ===")
Mo = qu*S1.W*.001*(S1.L*.001)**2 / 8 #kN-m, for each direction (N-S and E-W)
print(f"1. Total Statical Moment, Mo = {rd(Mo)}kN-m ({kNm_to_kipft(Mo)}kip-ft) for each direction.")
#Widths of Middle and Column Strip...
width_cs = rd(min(0.5*S1.L,0.5*S1.W)) # 8.4.1.5
print(f"2a. Width of Column Strip: {width_cs}mm") 
width_ms = rd(max(S1.L - width_cs, S1.W - width_cs))
print(f"2b. Width of Middle Strip: {width_ms}mm")

=== TOTAL STATICAL MOMENT, Mo ===
1. Total Statical Moment, Mo = 594.97kN-m (438.83kip-ft) for each direction.
2a. Width of Column Strip: 2865.0mm
2b. Width of Middle Strip: 4395.0mm


In [28]:
#Interior Panel Moments Distribution of Mo
print("=== Distribuition of Mo for Interior Panels (8.10.4.1) ===")
Mo_neg = rd(0.65*Mo) #at supports/ends of slab
Mo_pos = rd(0.35*Mo) #at midspan of slab
print(f" 1a. Interior Panel Moment (negative), Mo_neg = {Mo_neg}kNm.")
print(f" 1b. Interior Panel Moment (positive), Mo_pos = {Mo_pos}kNm.")
#Interior Panel, Column Strip, Table 8.10.5.1
Mo_neg_cs = rd(0.75*Mo_neg)
print(f"  2a. IPM_neg, Column Strip, Mo_neg_cs = {Mo_neg_cs}kNm.")
#Interior Panel, Middle Strip
Mo_neg_ms = rd(Mo_neg - Mo_neg_cs) #8.10.6
print(f"  2b. IPM_neg, Middle Strip, Mo_neg_ms = {Mo_neg_ms} kNm")
#Pos Mu, Column Strip, Table 8.10.5.5
Mo_pos_cs = rd(0.75*Mo_pos)
print(f"  2c. IPM_pos, Column Strip, Mo_pos_cs = {Mo_pos_cs} kNm")
Mo_pos_ms = rd(Mo_pos - Mo_pos_cs) #8.10.6
print(f"  2d. IPM_pos, Mid Strip, Mo_pos_ms = {Mo_pos_ms} kNm")

=== Distribuition of Mo for Interior Panels (8.10.4.1) ===
 1a. Interior Panel Moment (negative), Mo_neg = 386.73kNm.
 1b. Interior Panel Moment (positive), Mo_pos = 208.24kNm.
  2a. IPM_neg, Column Strip, Mo_neg_cs = 290.05kNm.
  2b. IPM_neg, Middle Strip, Mo_neg_ms = 96.68 kNm
  2c. IPM_pos, Column Strip, Mo_pos_cs = 156.18 kNm
  2d. IPM_pos, Mid Strip, Mo_pos_ms = 52.06 kNm


In [29]:
#End span Moments Distribution of Mo ACI318M-14 Table 8.10.4.2
print("=== Distribution of Mo for End Span with beams between all supports ===")
Mo_eneg = rd(0.16*Mo)
Mo_ineg = rd(0.70*Mo)
Mo_pos = rd(0.57*Mo)
print(f" End Span Panel Moment (negative exterior), Mo_eneg = {Mo_eneg}kNm.")
print(f" End Span Panel Moment (negative interior), Mo_ineg = {Mo_ineg}kNm.")
print(f" End Span Panel Moment (positive), Mo_epos = {Mo_pos}kNm.")
Mo_eneg_cs = rd(0.65*Mo_eneg)
print(f"    1. ESPM_eneg, Col Strip, Mo_eneg_cs = {Mo_eneg_cs}kNm")
Mo_ineg_cs = rd(0.75*Mo_ineg)
print(f"    2. ESPM_ineg, Col Strip, Mo_ineg_cs = {Mo_ineg_cs}kNm")
Mo_pos_cs = rd(0.75*Mo_pos)
print(f"    3. ESPM_pos, Column Strip, Mo_pos_cs = {Mo_pos_cs} kNm")
Mo_eneg_ms = rd(Mo_eneg - Mo_eneg_cs)
print(f"    1. ESPM_eneg, Mid Strip, Mo_eneg_ms = {Mo_eneg_ms}kNm")
Mo_ineg_ms = rd(Mo_ineg - Mo_ineg_cs)
print(f"    2. ESPM_ineg, Mid Strip, Mo_ineg_ms = {Mo_ineg_ms}kNm")
Mo_pos_ms = rd(Mo_pos - Mo_pos_cs)
print(f"    3. ESPM_pos, Mid Strip, Mo_pos_ms = {Mo_pos_ms} kNm")

=== Distribution of Mo for End Span with beams between all supports ===
 End Span Panel Moment (negative exterior), Mo_eneg = 95.19kNm.
 End Span Panel Moment (negative interior), Mo_ineg = 416.48kNm.
 End Span Panel Moment (positive), Mo_epos = 339.13kNm.
    1. ESPM_eneg, Col Strip, Mo_eneg_cs = 61.87kNm
    2. ESPM_ineg, Col Strip, Mo_ineg_cs = 312.36kNm
    3. ESPM_pos, Column Strip, Mo_pos_cs = 254.35 kNm
    1. ESPM_eneg, Mid Strip, Mo_eneg_ms = 33.32kNm
    2. ESPM_ineg, Mid Strip, Mo_ineg_ms = 104.12kNm
    3. ESPM_pos, Mid Strip, Mo_pos_ms = 84.78 kNm


In [30]:
#Strip / width
bw_supp = width_cs #mm
bw_mid = width_ms #mm
print(f"bw_mid = {bw_mid}mm | bw_supp = {width_cs}mm")

#RSB SPACING ALONG LONG DIR
#Moment for bottom reinf
LDir_mid_Mo_bot = Mo_pos_ms
LDir_sup_Mo_bot = Mo_pos_cs

#moment for top reinf
LDir_sup_Mo_top = Mo_neg_cs


bw_mid = 4395.0mm | bw_supp = 2865.0mm


In [31]:
#Material Properties
fc = 27       #concrete compressive strength, MPa
fy = 230        #steel yield strength, MPa
fyt = 230       #steel yield strength of ties, MPa
Es = 200000     #Steel Elasticity, GPa
print(f"==== MATERIAL PROPERTIES ====\nfc = {fc}MPa, fy = {fy}MPa, fyt = {fyt}MPa, ")

#Tensile Reinforcement Detail
d_main = 12     #diameter of main bars, mm
d_tie = 12      #diameter of tie bars, mm
n_tieleg = 2    #no. of tie legs
sc = 25         #steel cover

==== MATERIAL PROPERTIES ====
fc = 27MPa, fy = 230MPa, fyt = 230MPa, 


In [32]:
#MomentCapacity
import slab2w_capacity
print("=== LONG DIR: MIDSPAN BOT BARS ===")
slab2w_capacity.MomentCap(bw_mid,S1.thk,d_main,d_tie,sc,1,40,0,LDir_mid_Mo_bot)


=== LONG DIR: MIDSPAN BOT BARS ===


TypeError: MomentCap() takes 4 positional arguments but 9 were given